In [1]:
import sys
import os
import inspect
import numpy as np
import pandas
import pickle
from matplotlib import pyplot as plt
from scipy.stats import rankdata


currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
parentdir2 = os.path.dirname(parentdir)
parentdir3 = os.path.dirname(parentdir2)
parentdir4 = os.path.dirname(parentdir3)
sys.path.insert(0, parentdir) 
sys.path.insert(0, parentdir2)
sys.path.insert(0, parentdir3)
sys.path.insert(0, parentdir4) 


from bayesian_benchmarks.database_utils import Database
from bayesian_benchmarks.data import classification_datasets, _ALL_REGRESSION_DATATSETS, _ALL_CLASSIFICATION_DATATSETS


Bad key text.latex.unicode in file /home/stipendiater/mohamadi/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 112 ('text.latex.unicode : False # use "ucs" and "inputenc" LaTeX packages for handling')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.4/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.frameon in file /home/stipendiater/mohamadi/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 423 ('savefig.frameon : True')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.4/matplotlibrc.template
or from the matplotlib source distribution

Bad key nbagg.transparent in file /home/stipendiater/mohamadi/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 426 ('nbagg.transparent: True')
You probably need to get a

DATA_PATH /home/stipendiater/mohamadi/ProbAI/PapersWithCode/improvement/drbayes-master/experiments/uci_exps/bayesian_benchmarks/data/


In [2]:
ALL_DATATSETS = {}
ALL_DATATSETS.update(_ALL_REGRESSION_DATATSETS)
ALL_DATATSETS.update(_ALL_CLASSIFICATION_DATATSETS)
#from bayesian_benchmarks.data import regression_datasets

In [3]:
def rankarray(A):
    ranks = []
    for a in A:
        ranks.append(rankdata(a))
    return np.array(ranks)


def read_regression_classification(db_loc, fs, models_names, datasets, task):
    fields = ['dataset', 'N', 'D'] + [m[1] for m in models_names]

    results = {}
    for f in fs:
        results[f] = {'table':{f:[] for f in fields}, 'vals':[]}

    with Database(db_loc) as db:

        for dataset in datasets:
            for f in fs:
                results[f]['table']['dataset'].append(dataset[:10])
                results[f]['table']['N'].append(ALL_DATATSETS[dataset].N)
                results[f]['table']['D'].append(ALL_DATATSETS[dataset].D)

            row = {f:[] for f in fs}
            for model, name in models_names:
                res = db.read(task, fs, {'model':model, 
                                         'dataset':dataset})

                if len(res) == 0:
                    for f in fs:
                        results[f]['table'][name].append('')
                        row[f].append(np.nan)
                else:
                    print('{} {} {}'.format(model, dataset, len(res)))
                    for i, f in enumerate(fs):
                        L = [np.nan if l[i] is None else float(l[i]) for l in res]
                        m = np.nanmean(L)
                        std = np.nanstd(L) if len(L) > 1 else np.nan
                        if m < 1000 and m > -1000:
                            r = '{:.3f}({:.3f})'.format(m, std)
                            row[f].append(m)
                        else:
                            r = 'nan'
                            row[f].append(np.nan)

                        results[f]['table'][name].append(r)

            for f in fs:   
                results[f]['vals'].append(row[f])


    for f in fs:
        if 'unnormalized' not in f:
            vals = np.array(results[f]['vals'])

            avgs = np.nanmean(vals, 0)
            meds = np.nanmedian(vals, 0)
            rks = np.nanmean(rankarray(vals), 0)

            for s, n in [[avgs, 'avg'], [meds, 'median'], [rks, 'avg rank']]:
                results[f]['table']['dataset'].append(n)
                results[f]['table']['N'].append('')
                results[f]['table']['D'].append('')
                if task == 'classification':
                    results[f]['table']['K'].append('')
                for ss, name in zip(s, [m[1] for m in models_names]):
                    results[f]['table'][name].append('{:.3f}'.format(ss))
    
    return results, fields


models_names = [['RegNet', 'SGD'], ['RegNetNL_LP', 'NL'], ['RegNetpcaess', 'PCA+ESS'], ['RegNetpcavi', 'PCA+VI'],
               ['RegNetpcalow_rank_gaussian', 'SWAG']]
#regression_datasets = ['wilson_elevators', 'wilson_keggdirected', 'wilson_keggundirected', 'wilson_protein',
#		'wilson_pol', 'wilson_skillcraft'
#		]
regression_datasets = ['boston', 'concrete', 'energy', 'naval', 'yacht', 'kin8nm']

fs = ['test_loglik', 'test_rmse', 'test_loglik_unnormalized', 'test_rmse_unnormalized', 'test_calibration']

results, fields = read_regression_classification('results_small.db', fs, models_names, regression_datasets, 'regression')


print('normalized test loglikelihood')
print(pandas.DataFrame(results['test_loglik']['table'], columns=fields))

print('test loglikelihood')
print(pandas.DataFrame(results['test_loglik_unnormalized']['table'], columns=fields))


print('normalised test rmse')
print(pandas.DataFrame(results['test_rmse']['table'], columns=fields))

print('test rmse')
print(pandas.DataFrame(results['test_rmse_unnormalized']['table'], columns=fields))

print('test calibration')
print(pandas.DataFrame(results['test_calibration']['table'], columns=fields))

output_file = 'results_small.db'[:-3]
with open(output_file+'.pkl', 'wb') as f:
	pickle.dump({'results':results, 'fields':fields}, f, pickle.HIGHEST_PROTOCOL)

RegNet boston 382
RegNetpcaess boston 382
RegNetpcalow_rank_gaussian boston 382
RegNet concrete 382
RegNetpcaess concrete 382
RegNetpcalow_rank_gaussian concrete 380
RegNet naval 380
RegNetpcaess naval 380
RegNetpcalow_rank_gaussian naval 382
RegNet yacht 382
RegNetpcaess yacht 382
RegNetpcalow_rank_gaussian yacht 382
normalized test loglikelihood
    dataset      N   D            SGD     NL        PCA+ESS PCA+VI  \
0    boston    506  13  -0.559(0.000)         -0.551(0.000)          
1  concrete   1030   8  -0.296(0.000)         -0.212(0.000)          
2    energy    768   8                                               
3     naval  11934  14   1.101(0.000)          1.242(0.000)          
4     yacht    308   6   0.212(0.000)          0.205(0.000)          
5    kin8nm   8192   8                                               
6       avg                     0.114    nan          0.171    nan   
7    median                    -0.042    nan         -0.004    nan   
8  avg rank         

/home/stipendiater/mohamadi/.local/lib/python3.6/site-packages/ipykernel_launcher.py:55: RuntimeWarning: Mean of empty slice
/home/stipendiater/mohamadi/.local/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1114: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input)
